In [32]:
import pandas as pd
from gurobipy import Model, GRB
import numpy as np

In [33]:
# Load dataset
data_path = '/Users/mahinbindra/Downloads/price_response (1).csv'
df = pd.read_csv(data_path)

In [34]:
intercepts = df['Intercept'].to_numpy()
sensitivities = abs(df['Sensitivity'].to_numpy())  # Apply abs here
capacities = df['Capacity'].to_numpy()

In [35]:
# Create a Gurobi model
model = Model('Maximize Revenue')

In [36]:
# Add price variables for each product
prices = model.addVars(df.index, name='price', lb=0)

In [37]:
# Define the objective function (total revenue)
revenue = sum((intercepts[i] - sensitivities[i] * prices[i]) * prices[i] for i in df.index)
model.setObjective(revenue, GRB.MAXIMIZE)

In [38]:
# Add capacity constraints
for i in df.index:
    demand = intercepts[i] - sensitivities[i] * prices[i]
    model.addConstr(demand <= capacities[i], name=f'capacity_{i}')

In [39]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 9 columns and 9 nonzeros
Model fingerprint: 0xd1ce905f
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 9 rows and 9 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective 7.38368672e+08


In [40]:
# Print the optimal prices and total revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for i in df.index:
        print(f"{df.loc[i, 'Product']}: ${prices[i].X:.2f}")
    print(f"Total Revenue: ${model.ObjVal:.2f}")
else:
    print("No optimal solution found.")

Optimal Prices:
Line 1 Product 1: $383.85
Line 1 Product 2: $2296.50
Line 1 Product 3: $2351.88
Line 2 Product 1: $2050.30
Line 2 Product 2: $4160.71
Line 2 Product 3: $6813.66
Line 3 Product 1: $8138.45
Line 3 Product 2: $4498.42
Line 3 Product 3: $8558.94
Total Revenue: $738368671.96


In [41]:
# Set the intercepts and sensitivities for the two products
a1, a2, b1, b2 = intercepts[0], intercepts[1], sensitivities[0], sensitivities[1]

# Create a new model
m = Model("TechEssentials Pricing")

# Add decision variables
p1 = m.addVar(name="p1", lb=0)  # Price for Basic version
p2 = m.addVar(name="p2", lb=0)  # Price for Advanced version

# Set the objective function to maximize revenue
m.setObjective(p1 * (a1 - b1 * p1) + p2 * (a2 - b2 * p2), GRB.MAXIMIZE)

# Add constraints
m.addConstr(a1 - b1 * p1 >= 0, "DemandNonNegativityBasic")
m.addConstr(a2 - b2 * p2 >= 0, "DemandNonNegativityAdvanced")
m.addConstr(p2 - p1 >= 0.01, "PriceOrdering")

# Optimize the model
m.optimize()

# Print the optimal prices
if m.status == GRB.OPTIMAL:
    print(f"Optimal Price for Basic Version (p1): ${p1.X:.2f}")
    print(f"Optimal Price for Advanced Version (p2): ${p2.X:.2f}")
    print(f"Total Revenue: ${m.ObjVal:.2f}")  # Print the objective value
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x1d27bd7d
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [2e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 4e+04]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.57192109e+08  1.88361107e+09  8.82e+02 2.92e+02  1.

In [42]:
m = Model("TechEssentials Pricing Gradient Descent")

a1, a2, b1, b2 = intercepts[0], intercepts[1], sensitivities[0], sensitivities[1]

# Define the initial prices for both products
initial_prices = np.array([0, 0])

# Define the step size and stopping criterion for the gradient descent
step_size = 0.001
stopping_criterion = 1e-6

# Define the demand functions for the two products
def demand(p, a, b):
    return a - b * p

# Define the gradient of the objective function
def gradient(p):
    return np.array([
        a1 - 2 * b1 * p[0],  # Partial derivative with respect to p1
        a2 - 2 * b2 * p[1]   # Partial derivative with respect to p2
    ])

# Initialize the prices
prices = initial_prices

# Create the Gurobi model and variables outside of the loop
m = Model('Projection')
m.setParam('OutputFlag', 0) # Suppress Gurobi output

p1 = m.addVar(lb=0, name="p1")
p2 = m.addVar(lb=0, name="p2")

m.addConstr(a1 - b1 * p1 >= 0, "DemandNonNegativityBasic")
m.addConstr(a2 - b2 * p2 >= 0, "DemandNonNegativityAdvanced")
m.addConstr(p2 - p1 >= 0.01, "PriceOrdering")

iteration_counter = 0

# List to store revenue at each iteration
revenues = []

# Start the projected gradient descent algorithm
while True:
    iteration_counter += 1  # Increment the counter
    # Compute the gradient at the current prices
    grad = gradient(prices)
    
    # Take a step in the direction of the gradient
    new_prices = prices + step_size * grad
    
    # Update the model with the new objective function
    m.setObjective((p1 - new_prices[0])*(p1 - new_prices[0]) +
                   (p2 - new_prices[1])*(p2 - new_prices[1]),
                   GRB.MINIMIZE)

    # Optimize the model
    m.optimize()
    
    # Extract the projected prices
    projected_prices = np.array([p1.X, p2.X])

    # Calculate and store the revenue at the current prices
    current_revenue = projected_prices[0] * (a1 - b1 * projected_prices[0]) + projected_prices[1] * (a2 - b2 * projected_prices[1])
    revenues.append(current_revenue)
    
    # Check the stopping criterion
    if np.linalg.norm(prices - projected_prices) < stopping_criterion:
        break
    
    # Update the prices
    prices = projected_prices
    if iteration_counter % 15 == 0:  # Print every 15 iterations to track progress
        print(f'Iteration {iteration_counter}: Prices = {prices}, Revenue = {current_revenue}')

# Print the final prices and revenue
print("\n")
print(f'The model ran {iteration_counter} iterations.')
print("Optimal prices found:", prices)
print("Optimal revenue:", revenues[-1])

Iteration 15: Prices = [293.28898671 505.99254082], Revenue = 23400994.273436334
Iteration 30: Prices = [362.48310029 900.49865794], Revenue = 34099570.06101777
Iteration 45: Prices = [ 378.80770017 1208.08239466], Revenue = 40406756.52060907
Iteration 60: Prices = [ 382.65907635 1447.8955458 ], Revenue = 44229856.07962936
Iteration 75: Prices = [ 383.56771108 1634.8701501 ], Revenue = 46553241.22020462
Iteration 90: Prices = [ 383.78208045 1780.64823811], Revenue = 47965551.28245162
Iteration 105: Prices = [ 383.83265548 1894.30672677], Revenue = 48824067.20523352
Iteration 120: Prices = [ 383.84458738 1982.92259643], Revenue = 49345943.44097765
Iteration 135: Prices = [ 383.84740241 2052.01354113], Revenue = 49663182.583457835
Iteration 150: Prices = [ 383.84806654 2105.88152953], Revenue = 49856026.53439989
Iteration 165: Prices = [ 383.84822323 2147.88066832], Revenue = 49973252.89297947
Iteration 180: Prices = [ 383.84826019 2180.62604661], Revenue = 50044512.68142823
Iteration 19

In [43]:
model = Model('Maximize Revenue Within Line Constraints')
# Add price variables for each product
prices = model.addVars(len(df), name='price', lb=0)

# Define the objective function (total revenue)
revenue = sum((intercepts[i] - sensitivities[i] * prices[i]) * prices[i] for i in range(len(df)))
model.setObjective(revenue, GRB.MAXIMIZE)

# Add capacity constraints
for i in range(len(df)):
    demand = intercepts[i] - sensitivities[i] * prices[i]
    model.addConstr(demand <= capacities[i], name=f'capacity_{i}')


#Add constraints for price ordering within each product line
num_products_per_line = 3
for i in range(0, len(df), num_products_per_line):
    model.addConstr(prices[i+1] - prices[i] >= 0.01, name=f'price_order_{i}_basic_advanced')
    model.addConstr(prices[i+2] - prices[i+1] >= 0.01, name=f'price_order_{i}_advanced_premium')


# Optimize the model
model.optimize()

# Print the optimal prices and total revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for i in range(len(df)):
        print(f"{df.loc[i, 'Product']}: ${prices[i].X:.2f}")
    print(f"Total Revenue: ${model.ObjVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15 rows, 9 columns and 21 nonzeros
Model fingerprint: 0xc90e07c7
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 5e+04]
Presolve removed 9 rows and 0 columns
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.34101424e+08  1.18618541e+09  5.00e+02 2.21e+02 

In [44]:
# Create model
model = Model('Maximize Revenue Across Line Constraints')

# Add price variables for each product
prices = model.addVars(len(df), name='price', lb=0)

# Define the objective function (total revenue)
revenue = sum((intercepts[i] - sensitivities[i] * prices[i]) * prices[i] for i in range(len(df)))
model.setObjective(revenue, GRB.MAXIMIZE)

# Add capacity constraints
for i in range(len(df)):
    demand = intercepts[i] - sensitivities[i] * prices[i]
    model.addConstr(demand <= capacities[i], name=f'capacity_{i}')

#Add constraints for price ordering within each product line
num_products_per_line = 3
for i in range(0, len(df), num_products_per_line):
    model.addConstr(prices[i+1] - prices[i] >= 0.01, name=f'price_order_{i}_basic_advanced')
    model.addConstr(prices[i+2] - prices[i+1] >= 0.01, name=f'price_order_{i}_advanced_premium')


# Add constraints for price ordering across product lines for the same version
model.addConstr(prices[3] - prices[0]>= 0.01, name='price_order_basic_1')  # Product 1 Basic must be priced at least $0.01 lower than Product 2 Basic
model.addConstr(prices[6] - prices[3]>= 0.01, name='price_order_basic_2')  # Product 2 Basic must be priced at least $0.01 lower than Product 3 Basic
model.addConstr(prices[4] - prices[1]>= 0.01, name='price_order_advanced_1')  # Product 1 Advanced must be priced at least $0.01 lower than Product 2 Advanced
model.addConstr(prices[7] - prices[4]>= 0.01, name='price_order_advanced_2')  # Product 2 Advanced must be priced at least $0.01 lower than Product 3 Advanced
model.addConstr(prices[5] - prices[2]>= 0.01, name='price_order_premium_1')  # Product 1 Premium must be priced at least $0.01 lower than Product 2 Premium
model.addConstr(prices[8] - prices[5]>= 0.01, name='price_order_premium_2')  # Product 2 Premium must be priced at least $0.01 lower than Product 3 Premium

# Optimize the model
model.optimize()

# Print the optimal prices and total revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for i in range(len(df)):
        print(f"{df.loc[i, 'Product']}: ${prices[i].X:.2f}")
    print(f"Total Revenue: ${model.ObjVal:.2f}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 9 columns and 33 nonzeros
Model fingerprint: 0xc4309bf1
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 5e+04]
Presolve removed 9 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.34101813e+08  1.18618630e+09  5.00e+02 1.83e+02